In [1]:
model_name_mapping = {
    0: "gemini",
    1:"gpt-4o",
    2: "llama"
}
question_types = [
    "Computer Science & Programming",
    "Mathematics & Statistics",
    "Science & Engineering",
    "Business & Finance",
    "Writing & Communication",
    "Social & Daily Life",
    "Others"
]

In [2]:
gemini_data_path = "/data2/xianglin/data/preference_leakage/UltraFeedback_sampled_30000_gemini_LlamaFactory.json"
gpt4_data_path = "/data2/xianglin/data/preference_leakage/UltraFeedback_sampled_30000_gpt4_LlamaFactory.json"
llama_data_path = "/data2/xianglin/data/preference_leakage/UltraFeedback_sampled_30000_llama_LlamaFactory.json"

import json
gemini_data = json.load(open(gemini_data_path))
gpt4_data = json.load(open(gpt4_data_path))
llama_data = json.load(open(llama_data_path))

len(gemini_data), len(gpt4_data), len(llama_data)


(30000, 29987, 30000)

In [3]:
question_type_data_path = "/home/xianglin/git_space/llm-as-a-judge-analysis/analysis/dataset_categorization/ultrafeedback_categorization.json"

with open(question_type_data_path, "r") as f:
    question_type_data = json.load(f)

question_type_data[0]

# process question type data
question_type_mapping = {}
for item in question_type_data:
    if 'question' in item.keys() and 'categorization' in item.keys() and 'question category' in item['categorization'].keys():
        question_type_mapping[item['question']] = item['categorization']['question category']


In [4]:
# merge
count = 0
for i in range(len(gemini_data)):
    instruction = gemini_data[i]['instruction']
    qt = question_type_mapping.get(instruction, None)
    if qt is not None:
        gemini_data[i]['question_type'] = qt
        count += 1
print(f"Total {count} question types found")


Total 29997 question types found


In [5]:
# merge
count = 0
for i in range(len(gpt4_data)):
    instruction = gpt4_data[i]['instruction']
    qt = question_type_mapping.get(instruction, None)
    if qt is not None:
        gpt4_data[i]['question_type'] = qt
        count += 1
print(f"Total {count} question types found")

Total 29984 question types found


In [6]:
# merge
count = 0
for i in range(len(llama_data)):
    instruction = llama_data[i]['instruction']
    qt = question_type_mapping.get(instruction, None)
    if qt is not None:
        llama_data[i]['question_type'] = qt
        count += 1
print(f"Total {count} question types found")

Total 29997 question types found


In [13]:
from transformers import pipeline

OUTPUT_DIR = "/data2/xianglin/data/preference_leakage/bert_classifier_results"
pipe = pipeline("text-classification", model=f"{OUTPUT_DIR}/best_model", tokenizer=f"{OUTPUT_DIR}/best_model", max_length=512, truncation=True)
new_text = "This text might be from the second model."
result = pipe(new_text)
print(f"\nPrediction for '{new_text}': {result}")

Device set to use cuda:0



Prediction for 'This text might be from the second model.': [{'label': 'LABEL_0', 'score': 0.7317314147949219}]


In [2]:
print("\nLoading model for prediction example...")

import joblib

MODEL_SAVE_PATH = "/data2/xianglin/data/preference_leakage/bow_classifier_LogisticRegression.joblib"
VECTORIZER_SAVE_PATH = "/data2/xianglin/data/preference_leakage/bow_vectorizer_tfidf_ngram2_maxf20000.joblib"
loaded_classifier = joblib.load(MODEL_SAVE_PATH)
loaded_vectorizer = joblib.load(VECTORIZER_SAVE_PATH)
new_texts = ["This sounds like Model A.", "A different kind of text from B."]
new_texts_vec = loaded_vectorizer.transform(new_texts)
predictions = loaded_classifier.predict(new_texts_vec)
print(f"Predictions for new texts: {predictions}")


Loading model for prediction example...
Predictions for new texts: [1 0]


In [4]:
print("\nLoading model for extracted feature prediction example...")
from src.extract_features import extract_feature_vector

MODEL_SAVE_PATH = "/data2/xianglin/data/preference_leakage/extracted_feature_bow_classifier_LogisticRegression.joblib"
loaded_classifier = joblib.load(MODEL_SAVE_PATH)
new_texts = ["This sounds like Model A.", "A different kind of text from B."]
new_texts_vec = [extract_feature_vector(text) for text in new_texts]
predictions = loaded_classifier.predict(new_texts_vec)
print(f"Predictions for new texts: {predictions}")


Loading model for extracted feature prediction example...
Predictions for new texts: [1 1]


/home/xianglin/miniconda3/envs/llm/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
